# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [2]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [3]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('C:/Users/Usuario/Documents/P/M/dmeyf/fuente/datasets/competencia_01_crudo.csv')

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [4]:
%sql select count(*) from competencia_01

,count_star()
0,491063


In [5]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [6]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [7]:
%%sql
select t1.numero_de_cliente 
from competencia_01 t1
WHERE (t1.foto_mes = 202103)
LIMIT 5

,numero_de_cliente
0,29183981
1,29184630
2,29185433
3,29185587
4,29185646


Nota: esto siempre retorna todo NULL en la tabla derecha.

In [9]:
%%sql

select t1.numero_de_cliente, t1.foto_mes, t2.foto_mes
from competencia_01 t1 
LEFT JOIN competencia_01 t2 
ON (t1.foto_mes = 0)
AND (t1.numero_de_cliente = t2.numero_de_cliente)
LIMIT 5

,numero_de_cliente,foto_mes,foto_mes_2
0,29183981,202103,NaN
1,29184630,202103,NaN
2,29185433,202103,NaN
3,29185587,202103,NaN
4,29185646,202103,NaN


Si se quiere mantener la tabla original:

In [15]:
%%sql

create or replace table competencia_01_m as 

select t0.*, 
case
    WHEN (t0.foto_mes != 202103) THEN NULL 
    WHEN (t1.foto_mes is NULL) THEN 'BAJA+1'
    WHEN (t2.foto_mes is NULL) THEN 'BAJA+2'
    ELSE 'CONTINUA'
END as cternaria
from competencia_01 t0
LEFT JOIN competencia_01 t1
ON (t0.numero_de_cliente = t1.numero_de_cliente)
AND (t0.foto_mes + 1 = t1.foto_mes)
LEFT JOIN competencia_01 t2
ON (t0.numero_de_cliente = t2.numero_de_cliente)
AND (t0.foto_mes + 2 = t2.foto_mes)

,Success


Si se quiere una tabla separada para el primer mes:

In [14]:
%%sql

create or replace table competencia_01_m as 

select t0.*, 
case
    WHEN (t1.foto_mes is NULL) THEN 'BAJA+1'
    WHEN (t2.foto_mes is NULL) THEN 'BAJA+2'
    ELSE 'CONTINUA'
END as cternaria
from competencia_01 t0
LEFT JOIN competencia_01 t1
ON (t0.numero_de_cliente = t1.numero_de_cliente)
AND (202104 = t1.foto_mes)
LEFT JOIN competencia_01 t2
ON (t0.numero_de_cliente = t2.numero_de_cliente)
AND (202105 = t2.foto_mes)
WHERE t0.foto_mes = 202103

,Success


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [16]:
%%sql
select 
    cternaria, 
    count(*) as cant,
    cant/(SELECT count(*) from competencia_01_m) as prop_tot,
    cant/(SELECT count(*) from competencia_01_m WHERE cternaria is not NULL) as prop_m0
from competencia_01_m
group by cternaria

,cternaria,cant,prop_tot,prop_m0
0,NaN,327739,0.667407,2.006680
1,CONTINUA,161311,0.328493,0.987675
2,BAJA+2,963,0.001961,0.005896
3,BAJA+1,1050,0.002138,0.006429


Para guardar solo los datos del primer mes:

In [17]:
%%sql
create or replace table competencia_01_m as 

SELECT t.*
FROM competencia_01_m t
WHERE t.cternaria IS NOT NULL

,Success


In [22]:
%%sql

SELECT count(*) = (SELECT count(*) FROM competencia_01 t WHERE t.foto_mes = 202103) es_valida
FROM competencia_01_m
LIMIT 1

,es_valida
0,True


Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [41]:
%%sql
COPY competencia_01_m TO 'C:/Users/Usuario/Documents/P/M/dmeyf/fuente/datasets/competencia_01.csv' (FORMAT CSV, HEADER)

,Success


## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo

In [23]:
%%sql
create or replace table t_ej2 as 

SELECT *
FROM read_csv_auto('C:/Users/Usuario/Documents/P/M/dmeyf/fuente/datasets/ejercicio_target.csv')

,Success


In [25]:
%%sql

SELECT *
FROM t_ej2
LIMIT 5

,periodo,nombre
0,202110,Isaac Valverde
1,202110,Santiago Córdova Tercero
2,202110,Jerónimo Ocasio
3,202110,Isabel Terrazas Hijo
4,202110,Diego Caballero


In [27]:
%%sql

SELECT t.periodo, COUNT(*) cant
FROM t_ej2 t
GROUP BY t.periodo

,periodo,cant
0,202110,99
1,202111,110
2,202112,120
3,202201,130
4,202202,136
5,202203,144


In [34]:
%%sql

create or replace table t_ej2_m as 

select t0.*, 
case
    WHEN (t0.periodo != 202110) THEN NULL 
    WHEN (t1.periodo is NULL) THEN 'BAJA+1'
    WHEN (t2.periodo is NULL) THEN 'BAJA+2'
    ELSE 'CONTINUA'
END as cternaria_202110,
case
    WHEN (t0.periodo != 202111) THEN NULL 
    WHEN (t1.periodo is NULL) THEN 'BAJA+1'
    WHEN (t2.periodo is NULL) THEN 'BAJA+2'
    ELSE 'CONTINUA'
END as cternaria_202111,
case
    WHEN (t0.periodo != 202112) THEN NULL 
    WHEN (t1.periodo is NULL) THEN 'BAJA+1'
    WHEN (t2.periodo is NULL) THEN 'BAJA+2'
    ELSE 'CONTINUA'
END as cternaria_202112,
case
    WHEN (t0.periodo != 202201) THEN NULL 
    WHEN (t1.periodo is NULL) THEN 'BAJA+1'
    WHEN (t2.periodo is NULL) THEN 'BAJA+2'
    ELSE 'CONTINUA'
END as cternaria_202201
from t_ej2 t0
LEFT JOIN t_ej2 t1
ON (t0.nombre = t1.nombre)
AND (t0.periodo + 1 = t1.periodo)
LEFT JOIN t_ej2 t2
ON (t0.nombre = t2.nombre)
AND (t0.periodo + 2 = t2.periodo)

,Success


In [36]:
%%sql

SELECT *
FROM t_ej2_m
WHERE periodo=202110
LIMIT 5

,periodo,nombre,cternaria_202110,cternaria_202111,cternaria_202112,cternaria_202201
0,202110,Isaac Valverde,CONTINUA,NaN,NaN,NaN
1,202110,Santiago Córdova Tercero,CONTINUA,NaN,NaN,NaN
2,202110,Jerónimo Ocasio,CONTINUA,NaN,NaN,NaN
3,202110,Isabel Terrazas Hijo,CONTINUA,NaN,NaN,NaN
4,202110,Isabel Barajas,CONTINUA,NaN,NaN,NaN


In [38]:
%%sql

SELECT t.cternaria_202110, COUNT(*)
FROM t_ej2_m t
WHERE periodo=202110
GROUP BY cternaria_202110

,cternaria_202110,count_star()
0,CONTINUA,80
1,BAJA+2,11
2,BAJA+1,8
